# 제품 이상여부 판별 프로젝트

## 1. 데이터 불러오기

### 필수 라이브러리

In [28]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import xgboost as xgb
import shap
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

In [29]:
ROOT_DIR = "data"
RANDOM_STATE = 110
# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

In [30]:
drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[column].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)

### 전처리 함수

In [31]:
#전처리 함수

# 파트 1: 전처리 초기 단계
def preprocess_initial(df):
    # 고유값이 하나인 컬럼 찾기
    unique_value_1_columns = [column for column in df.columns if df[column].nunique() == 1]

    # 모든 행의 값이 다른 컬럼 찾기
    row_count = len(df)
    matching_row_columns = [column for column in df.columns if df[column].value_counts().size == row_count]

    # 고유값, 혼합값 피쳐 제거
    df.drop(columns=unique_value_1_columns, inplace=True)
    df.drop(columns=matching_row_columns, inplace=True)
    return df

# 파트 2: 유사한 피쳐 제거
def reduce_dataframe(df):
    def get_value_counts_ratio(series):
        value_counts = series.value_counts(normalize=True)
        return value_counts.sort_values().values

    # 고유값 매핑 확인 함수
    def check_value_mapping(df, col1, col2):
        unique_values_1 = df[col1].unique()
        unique_values_2 = df[col2].unique()

        if len(unique_values_1) != len(unique_values_2):
            return False

        value_mapping = {}
        for val1 in unique_values_1:
            corresponding_values = df[df[col1] == val1][col2].unique()
            if len(corresponding_values) != 1:
                return False
            value_mapping[val1] = corresponding_values[0]

        for val1 in unique_values_1:
            ratio_1 = (df[col1] == val1).mean()
            ratio_2 = (df[col2] == value_mapping[val1]).mean()
            if ratio_1 != ratio_2:
                return False

        return True

    def compare_all_features(df):
        ratios = {column: get_value_counts_ratio(df[column]) for column in df.columns}
        similar_columns_dict = {column: [] for column in df.columns}

        # 고유값 비율이 같은 열들 찾기
        columns = list(ratios.keys())
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                if np.array_equal(ratios[columns[i]], ratios[columns[j]]):
                    similar_columns_dict[columns[i]].append(columns[j])

        # 고유값 비율이 같고 매핑도 동일한 피쳐들 찾기
        comparisons = []
        for key, values in similar_columns_dict.items():
            for value in values:
                if check_value_mapping(df, key, value):
                    comparisons.append((key, value))

        if comparisons:
            print("다음 피쳐 쌍은 고유값 비율과 양상이 동일합니다:")
            for base, compare in comparisons:
                print(f"'{base}'와(과) '{compare}'")
        else:
            print("모든 피쳐의 고유값 비율과 양상이 동일하지 않습니다.")

        return comparisons

    comparisons = compare_all_features(df)
    columns_to_remove = set()
    for _, col_to_remove in comparisons:
        columns_to_remove.add(col_to_remove)

    df = df.drop(columns=columns_to_remove)
    global df_columns
    df_columns = df.columns
    return df

# 파트 3: 통계 계산 및 추가 전처리
def finalize_preprocessing(df):

    # 장비별 Collect Result 컬럼들 식별
    collect_result_columns_dam = [col for col in df.columns if 'Collect Result' in col and 'Dam' in col]
    collect_result_columns_autoclave = [col for col in df.columns if 'Collect Result' in col and 'AutoClave' in col]
    collect_result_columns_fill1 = [col for col in df.columns if 'Collect Result' in col and 'Fill1' in col]
    collect_result_columns_fill2 = [col for col in df.columns if 'Collect Result' in col and 'Fill2' in col]

    # Dam 장비의 평균, 표준편차, 합계 계산
    df['Collect_Result_Mean_Dam'] = df[collect_result_columns_dam].astype(float).mean(axis=1)
    df['Collect_Result_Std_Dam'] = df[collect_result_columns_dam].astype(float).std(axis=1)
    df['Collect_Result_Sum_Dam'] = df[collect_result_columns_dam].astype(float).sum(axis=1)

    # AutoClave 장비의 평균, 표준편차, 합계 계산
    df['Collect_Result_Mean_AutoClave'] = df[collect_result_columns_autoclave].astype(float).mean(axis=1)
    df['Collect_Result_Std_AutoClave'] = df[collect_result_columns_autoclave].astype(float).std(axis=1)
    df['Collect_Result_Sum_AutoClave'] = df[collect_result_columns_autoclave].astype(float).sum(axis=1)

    # Fill1 장비의 평균, 표준편차, 합계 계산
    df['Collect_Result_Mean_Fill1'] = df[collect_result_columns_fill1].astype(float).mean(axis=1)
    df['Collect_Result_Std_Fill1'] = df[collect_result_columns_fill1].astype(float).std(axis=1)
    df['Collect_Result_Sum_Fill1'] = df[collect_result_columns_fill1].astype(float).sum(axis=1)

    # Fill2 장비의 평균, 표준편차, 합계 계산
    df['Collect_Result_Mean_Fill2'] = df[collect_result_columns_fill2].astype(float).mean(axis=1)
    df['Collect_Result_Std_Fill2'] = df[collect_result_columns_fill2].astype(float).std(axis=1)
    df['Collect_Result_Sum_Fill2'] = df[collect_result_columns_fill2].astype(float).sum(axis=1)

    return df

# train 전처리 함수
def preprocess_train_dataframe(df):
    df = preprocess_initial(df)
    df = reduce_dataframe(df)
    return df

# test 전처리 함수
def preprocess_test_dataframe(df):
    df = preprocess_initial(df)
    return df

In [ ]:
df_train_pre = preprocess_train_dataframe(train_data)

In [33]:
features = df_train_pre.columns

In [34]:
# # 컬럼 목록을 사용자가 제공한 순서에 맞춰 재정렬
# columns_in_order = [
#     # 시간 관련(Time-related) 변수
#     'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
#     'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
#     'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',

#     'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
#     'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
#     'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
    
#     '1st Pressure 1st Pressure Unit Time_AutoClave',
#     '2nd Pressure Unit Time_AutoClave',
#     '3rd Pressure Unit Time_AutoClave',
#     'Machine Tact time Collect Result_Dam',
#     'Machine Tact time Collect Result_Fill1',
#     'Machine Tact time Collect Result_Fill2',

#     # 속도 관련(Speed-related) 변수
#     'CURE SPEED Collect Result_Dam',
#     'CURE SPEED Collect Result_Fill2',
#     'Stage1 Circle1 Distance Speed Collect Result_Dam',
#     'Stage1 Circle2 Distance Speed Collect Result_Dam',
#     'Stage1 Line1 Distance Speed Collect Result_Dam',
#     'Stage1 Line2 Distance Speed Collect Result_Dam',
#     'Stage1 Line4 Distance Speed Collect Result_Dam',
#     'Stage2 Circle1 Distance Speed Collect Result_Dam',
#     'Stage2 Circle2 Distance Speed Collect Result_Dam',
#     'Stage2 Line2 Distance Speed Collect Result_Dam',
#     'Stage2 Line3 Distance Speed Collect Result_Dam',
#     'Stage2 Line4 Distance Speed Collect Result_Dam',
#     'Stage3 Circle1 Distance Speed Collect Result_Dam',
#     'Stage3 Circle2 Distance Speed Collect Result_Dam',
#     'Stage3 Line1 Distance Speed Collect Result_Dam',
#     'Stage3 Line2 Distance Speed Collect Result_Dam',
#     'Stage3 Line4 Distance Speed Collect Result_Dam',
#     'DISCHARGED SPEED OF RESIN Collect Result_Dam',
#     'DISCHARGED SPEED OF RESIN Collect Result_Fill1',

#     # 볼륨 관련(Volume-related) 변수
#     'Dispense Volume(Stage1) Collect Result_Dam',
#     'Dispense Volume(Stage2) Collect Result_Dam',
#     'Dispense Volume(Stage3) Collect Result_Dam',
#     'Dispense Volume(Stage2) Collect Result_Fill1',

#     # 좌표 관련(Coordinate-related) 변수
#     'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
#     'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
#     'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',

#     'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
#     'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
#     'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',

#     'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
#     'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',

#     'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',

#     'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',

#     'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
#     'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
    
#     'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
#     'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
#     'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
#     'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
    

#     # 포지션 관련(Position-related) 변수
#     'HEAD Standby Position X Collect Result_Dam',
#     'HEAD Standby Position Y Collect Result_Dam',
#     'Head Clean Position Z Collect Result_Dam',
#     'Head Purge Position X Collect Result_Dam',
#     'Head Purge Position Z Collect Result_Dam',
#     'Head Zero Position Y Collect Result_Dam',
#     'Head Zero Position Z Collect Result_Dam',
#     'HEAD Standby Position X Collect Result_Fill1',
#     'Head Purge Position Z Collect Result_Fill1',
#     'CURE END POSITION X Collect Result_Fill2',
#     'CURE END POSITION Z Collect Result_Fill2',
#     'CURE STANDBY POSITION Z Collect Result_Fill2',
#     'HEAD Standby Position X Collect Result_Fill2',

#     # 압력 관련(Pressure-related) 변수
#     '1st Pressure Collect Result_AutoClave',
#     '2nd Pressure Collect Result_AutoClave',
#     '3rd Pressure Collect Result_AutoClave',

#     # 온도 관련(Temperature-related) 변수
#     'Chamber Temp. Collect Result_AutoClave',
#     'Chamber Temp. Unit Time_AutoClave',
#     'Chamber Temp. Judge Value_AutoClave',

#     # 두께 관련(Thickness-related) 변수
#     'THICKNESS 1 Collect Result_Dam',
#     'THICKNESS 2 Collect Result_Dam',
#     'THICKNESS 3 Collect Result_Dam',

#     # 기타 변수(Miscellaneous Variables)
#     'PalletID Collect Result_Dam',
#     'Production Qty Collect Result_Dam',
#     'Receip No Collect Result_Dam',
#     'WorkMode Collect Result_Dam',
#     'PalletID Collect Result_Fill1',
#     'Production Qty Collect Result_Fill1',
#     'Receip No Collect Result_Fill1',
#     'WorkMode Collect Result_Fill1',
#     'PalletID Collect Result_Fill2',
#     'Production Qty Collect Result_Fill2',
#     'Receip No Collect Result_Fill2',
#     'WorkMode Collect Result_Fill2',
#     'Equipment_Dam',
#     'Model.Suffix_Dam',
#     'Workorder_Dam',
#     'Equipment_Fill1',
#     'Equipment_Fill2',
#     'target'
# ]

# df_reordered = df_train_pre[columns_in_order]

In [35]:
df_order = df_train_pre.copy()

## 2. 데이터 전처리

### 언더 샘플링 

In [36]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = df_order[df_order["target"] == "Normal"]
df_abnormal = df_order[df_order["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(
    n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE
)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
dtype: int64

In [37]:
def preprocess_coordinates(df, columns_to_replace, columns_to_fill):
    # 1. 특정 값('OK')을 NaN으로 변환
    for column in columns_to_replace:
        df[column].replace('OK', np.nan, inplace=True)
    
    # 2. 숫자로 변환할 수 없는 데이터를 NaN으로 변환
    for column in columns_to_replace:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    # 3. 결측치를 처리하기 위한 최빈값 계산 및 채우기
    for column in columns_to_fill:
        mode_value = df[column].mode()[0]
        df[column].fillna(mode_value, inplace=True)
    
    return df

In [38]:
# 결측치 열

# 'OK' 값을 NaN으로 변환하고 결측치 처리할 열 리스트
columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 결측치를 처리할 열 리스트
columns_to_fill = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]


In [39]:
# 결측치 함수
df_concat = preprocess_coordinates(df_concat, columns_to_replace, columns_to_fill)

### 피쳐 생성 함수

In [40]:
def feature_engineering(df):
    # 1. 시간 관련 피처 생성
    df['Resin_Time_Diff_Stage1_2_Dam'] = df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    df['Resin_Time_Diff_Stage1_3_Dam'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    df['Resin_Time_Diff_Stage2_3_Dam'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
    df['Resin_Time_Diff_Stage1_2_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
    df['Resin_Time_Diff_Stage1_3_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
    df['Resin_Time_Diff_Stage2_3_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']

    df['Machine_Tact_Time_Dam_Fill1_Diff'] = df['Machine Tact time Collect Result_Dam'] - df['Machine Tact time Collect Result_Fill1']
    df['Machine_Tact_Time_Fill1_Fill2_Diff'] = df['Machine Tact time Collect Result_Fill1'] - df['Machine Tact time Collect Result_Fill2']

    # 2. 속도 관련 피처 생성
    df['CURE_SPEED_Dam_Fill_Diff'] = df['CURE SPEED Collect Result_Dam'] - df['CURE SPEED Collect Result_Fill2']
    df['Stage1_Circle_Speed_Diff_Dam'] = df['Stage1 Circle2 Distance Speed Collect Result_Dam'] - df['Stage1 Circle1 Distance Speed Collect Result_Dam']
    df['Stage2_Circle_Speed_Diff_Dam'] = df['Stage2 Circle2 Distance Speed Collect Result_Dam'] - df['Stage2 Circle1 Distance Speed Collect Result_Dam']
    df['Stage3_Circle_Speed_Diff_Dam'] = df['Stage3 Circle2 Distance Speed Collect Result_Dam'] - df['Stage3 Circle1 Distance Speed Collect Result_Dam']

    df['Stage1_Line21_Speed_Diff_Dam'] = df['Stage1 Line2 Distance Speed Collect Result_Dam'] - df['Stage1 Line1 Distance Speed Collect Result_Dam']
    df['Stage1_Line41_Speed_Diff_Dam'] = df['Stage1 Line4 Distance Speed Collect Result_Dam'] - df['Stage1 Line1 Distance Speed Collect Result_Dam']

    df['Stage2_Line_Speed_Diff_Dam'] = df['Stage2 Line3 Distance Speed Collect Result_Dam'] - df['Stage2 Line2 Distance Speed Collect Result_Dam']
    df['Stage3_Line_Speed_Diff_Dam'] = df['Stage3 Line2 Distance Speed Collect Result_Dam'] - df['Stage3 Line1 Distance Speed Collect Result_Dam']

    # 3. 볼륨 관련 피처 생성
    df['Dispense_Volume_Ratio_Stage1_2_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] / df['Dispense Volume(Stage2) Collect Result_Dam']
    df['Dispense_Volume_Ratio_Stage2_3_Dam'] = df['Dispense Volume(Stage2) Collect Result_Dam'] / df['Dispense Volume(Stage3) Collect Result_Dam']

    # 4. 좌표 관련 피처 생성
    df['Head_X_Coord_Diff_Stage2_3_Dam'] = df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam']
    df['Head_Y_Coord_Diff_Stage1_2_Dam'] = df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam']
    df['Head_Z_Coord_Diff_Stage1_2_Dam'] = df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']

    df['Head_X_Coord_Diff_Stage2_3_Fill1'] = df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1']
    df['Head_Y_Coord_Diff_Stage1_2_Fill1'] = df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1']
    df['Head_Z_Coord_Diff_Stage1_2_Fill1'] = df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1']

    # 5. 포지션 관련 피처 생성
    df['Head_Z_Clean_Purge_Diff_Dam'] = df['Head Clean Position Z Collect Result_Dam'] - df['Head Purge Position Z Collect Result_Dam']
    df['Head_X_Standby_Diff_Dam_Fill1'] = df['HEAD Standby Position X Collect Result_Dam'] - df['HEAD Standby Position X Collect Result_Fill1']
    df['Cure_End_Position_Diff_Fill2'] = df['CURE END POSITION X Collect Result_Fill2'] - df['CURE END POSITION Z Collect Result_Fill2']

    # 6. 압력 관련 피처 생성
    df['Pressure_Diff_1st_2nd_AutoClave'] = df['1st Pressure Collect Result_AutoClave'] - df['2nd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_2nd_3rd_AutoClave'] = df['2nd Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']

    # 8. 두께 관련 피처 생성
    df['Thickness_Diff_1_2'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 2 Collect Result_Dam']
    df['Thickness_Diff_2_3'] = df['THICKNESS 2 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']
    df['Thickness_Diff_1_3'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']

    # 비선형 조합 피처 생성
    df['Resin_Speed_Product_Dam'] = df['Resin_Time_Diff_Stage1_2_Dam'] * df['CURE_SPEED_Dam_Fill_Diff']
    df['Thickness_Diff_1_2_Squared'] = df['Thickness_Diff_1_2'] ** 2

    # 유클리드 거리 피처 생성 (좌표 관련)
    df['Head_Coord_Distance_Stage1_2_Dam'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']) ** 2
    )

    # 레진 방출 시간의 총합
    df['Total_Discharge_Time_Dam'] = (
        df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam'] +
        df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] +
        df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']
    )

    df['Total_Discharge_Time_Fill1'] = (
        df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1'] +
        df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1'] +
        df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']
    )

    # 단계 간의 시간 차이
    df['Stage1_2_Time_Difference_Dam'] = (
        df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] -
        df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    )

    # 기계 택트 타임 비율
    df['Tact_to_Discharge_Time_Ratio_Dam'] = (
        df['Machine Tact time Collect Result_Dam'] /
        df['Total_Discharge_Time_Dam']
    )

    # 총 볼륨
    df['Total_Volume_Dam'] = (
        df['Dispense Volume(Stage1) Collect Result_Dam'] +
        df['Dispense Volume(Stage2) Collect Result_Dam'] +
        df['Dispense Volume(Stage3) Collect Result_Dam']
    )

    df['Stage1_to_Total_Volume_Ratio_Dam'] = (
        df['Dispense Volume(Stage1) Collect Result_Dam'] /
        df['Total_Volume_Dam']
    )

    df['Stage2_to_Total_Volume_Ratio_Dam'] = (
        df['Dispense Volume(Stage2) Collect Result_Dam'] /
        df['Total_Volume_Dam']
    )

    df['Stage3_to_Total_Volume_Ratio_Dam'] = (
        df['Dispense Volume(Stage3) Collect Result_Dam'] /
        df['Total_Volume_Dam']
    )

    # 압력 증가율
    df['Pressure_Increment_Ratio_1_2'] = (
        (df['2nd Pressure Collect Result_AutoClave'] - df['1st Pressure Collect Result_AutoClave']) /
        df['1st Pressure Collect Result_AutoClave']
    )

    df['Pressure_Increment_Ratio_2_3'] = (
        (df['3rd Pressure Collect Result_AutoClave'] - df['2nd Pressure Collect Result_AutoClave']) /
        df['2nd Pressure Collect Result_AutoClave']
    )

    # 총 속도
    df['Total_Speed_Dam'] = (
        df['Stage1 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line4 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line4 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line4 Distance Speed Collect Result_Dam']
    )

    return df

In [41]:
# 피쳐 생성 함수 적용
df_concat = feature_engineering(df_concat)

# Scaler
numeric_columns = df_concat.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_concat[numeric_columns] = scaler.fit_transform(df_concat[numeric_columns])

In [42]:
train_x = df_concat.copy()

In [43]:
y = train_x.target
train_x.drop(columns = ['target'],inplace =True)
train_x = train_x.select_dtypes(exclude=['object'])

### SHAP

In [44]:
# 타겟 변수 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [45]:
# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(train_x, y_encoded, test_size=0.2, random_state=42)

# XGBoost 분류기 모델 초기화
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# F1 스코어 계산
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# SHAP 값 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# SHAP 값의 평균 절대값 계산
shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)
shap_importance = shap_values_df.abs().mean().sort_values(ascending=False)
shap_importance
# # SHAP 값 시각화
# shap.summary_plot(shap_values, X_test)

F1 Score: 0.5575


[22:22:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Production Qty Collect Result_Dam                             0.187820
Production Qty Collect Result_Fill2                           0.157497
PalletID Collect Result_Fill1                                 0.137194
Pressure_Increment_Ratio_2_3                                  0.134266
PalletID Collect Result_Dam                                   0.123823
Receip No Collect Result_Dam                                  0.112615
1st Pressure Collect Result_AutoClave                         0.107493
Pressure_Increment_Ratio_1_2                                  0.106232
Chamber Temp. Collect Result_AutoClave                        0.102297
Pressure_Diff_2nd_3rd_AutoClave                               0.097873
Receip No Collect Result_Fill1                                0.093268
Production Qty Collect Result_Fill1                           0.090195
PalletID Collect Result_Fill2                                 0.087541
Total_Discharge_Time_Fill1                                    0.085427
Head_Y

In [46]:
# SHAP 값이 0.01 이상인 피처만 선택
important_features = shap_importance[shap_importance > 0].index.tolist()

In [47]:
train_x_shap = train_x[important_features]

## 3. 모델

In [ ]:


catboost_model = CatBoostClassifier(verbose=0, random_state=42)
xgboost_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
randomforest_model = RandomForestClassifier(random_state=42)

In [ ]:
# 앙상블 모델 정의 (VotingClassifier)
voting_clf = VotingClassifier(
    estimators=[
        ('catboost', catboost_model),
        ('xgboost', xgboost_model),
        ('randomforest', randomforest_model)
    ],
    voting='soft'
)

# 앙상블 모델 학습
voting_clf.fit(train_x_shap, y)

## 4. 제출하기

### 테스트 데이터 예측

In [48]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

df_test = preprocess_test_dataframe(test_data)
df_test_pre = df_test.copy()

df_test_pre = df_test_pre[features]

df_test_pre.drop(columns ='target',inplace =  True)

In [49]:
#결측값 함수
df_test_pre = preprocess_coordinates(df_test_pre, columns_to_replace, columns_to_fill)

In [50]:
# 피쳐 생성 함수
df_test_pre = feature_engineering(df_test_pre)

numeric_columns = df_test_pre.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_test_pre[numeric_columns] = scaler.fit_transform(df_test_pre[numeric_columns])

df_test_pre = df_test_pre[important_features]

In [56]:
y_pred = voting_clf.predict(df_test_pre)

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**